<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center><font color=#555555><font size=6> Ingeniería Financiera </font> <br> <br>

<center><font color=#555555><font size=5> PAP </font> <br> <br> <br>

<center><font color=#555555><font size=5> I.F. Renatta García Sánchez </font> <br> <br>
   
<center> <font color= #555555> <font size = 4> Mayo 2020 </a> | <font color= #555555> <font size = 4> Repositorio: <a href='https://github.com/renattaGS/PAP-ML-ETICO'>Link</a></font>

<br>

---

<br>

<br>

<center> <font color= #555555> <font size = 6> Machine Learning ético </font> <br> <br> <font color= #555555>  </font>

# <font color= #555555> <font size = 5> Abstract </font>

<font color= #6B6B6B> <font size = 2> Este documento fue elaborado por Renatta García Sánchez, como trabajo para el proyecto de aplicación profesional: OPTIMIZACIÓN DE PROGRAMAS DE INVERSIÓN EN INTERMEDIARIOS FINANCIEROS, la cual es parte del curriculum de la licenciatura en Ingeniería Financiera, ofertada por la universidad **ITESO**. En el presente trabajo se plantea la respuesta a la siguiente pregunta: </font> 

<br>

# <font color= #6B6B6B> <font size = 6> 0. Librerías y Dependencias </font>

<hr style="border:0.02in solid gray"> </hr>

# <font color= #6B6B6B> <font size = 5> 0.1 Librerías </font> <font color= #555555>

Para correr este notebook es necesario tener instaladas y/o en el archivo _requirements.txt_ las siguientes librerías :

- pandas>=1.1.1 
- pandas-profiling>=2.11.0
- jupyter>=1.0.0
- numpy>= 1.19.1
- statsmodels>=0.11.1


# <font color= #6B6B6B> <font size = 5> 0.2 Dependencias </font> <font color= #555555>

Para correr este notebook es necesario contar con los siguientes archivos y dependencias externas:

- files/Datos_Techo_converted.xlsx : Datos sobre encuesta para el programa de hábitat de la organización Techo México. 

<br>

# <font color= #6B6B6B> <font size = 6> 1. Introducción </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

# <font color= #6B6B6B> <font size = 6> 2. Objetivos </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

# <font color= #6B6B6B> <font size = 6> 3. Datos </font>

<hr style="border:0.02in solid gray"> </hr>

Los datos con los que se trabajará corresponde las **Encuestas de Caracterización de Hogares (ECHS)** correspondientes a los años 2019 y 2020 de la ONG Techo México. Estas encuestas se realizan para determinar si una familia es candidata o no a recibir un apoyo del programa de viviendas. Para determinar esto, se contemplan un total de 97 variables. Con excepción del año de aplicación de la ECHS, todas las variables son de tipo discreto para proteger la información que estas contienen. 

# <font color= #6B6B6B> <font size = 6> 4. Procesos </font>

<hr style="border:0.02in solid gray"> </hr>

## Lectura de datos

In [12]:
import data

file_path='files/Datos_Techo_converted.xlsx'

data = data.lectura_datos(file_path, 'Datos Techo', 'p1')

reporte_profiling(data, 'Reporte_Datos_Techo.html') 


In [1]:
# aqui se incluye toda la secuencia de main.py.
# tomando los pasos en los que fuiste haciendo todos los calculos
# y poniendo cada uno como un subtitulo con markdown de nivel dos, así ## subtitulo de mi paso

<br>

# <font color= #6B6B6B> <font size = 6> 5. Conclusiones </font>

<hr style="border:0.02in solid gray"> </hr>

In [2]:
# en esta seccion es opcional incluir algun codigo para mostrar
# visualmente algun concepto o hallazgo relevante como conclusion

In [3]:
# En esta sección es opcional incluir código para mostrar/visualizar
# los datos con los que se trabajó en el laboratorio

<br>

# <font color= red> <font size = 6> 5. Pruebas (no incluir) </font>

<hr style="border:0.02in solid red"> </hr>

Las siguientes son una serie de pruebas para verificar que todo este funcionando bien, instaladas/cargadas las librerias, importar otros scripts, imprimir graficas de plotly.

In [4]:
import pandas as pd
import data as dt

In [5]:
# -- TEST 1 : 
# verify that the script is being read
print(dt.dict_test)

{'key_a': 'a', 'key_b': 'b'}


In [6]:
# -- TEST 2 :
# verify that installed pandas module works correctly
df_dict_test = pd.DataFrame(dt.dict_test, index=[0,1])
print(df_dict_test)

  key_a key_b
0     a     b
1     a     b


In [7]:
# -- TEST 3 :
# verify you can use plotly and visualize plots in jupyter notebook

import chart_studio.plotly as py  # various tools (jupyter offline print)
import plotly.graph_objects as go # plotting engine 

# example data
df = pd.DataFrame({'column_a': [1, 2, 3, 4, 5], 'column_b': [1, 2, 3, 4, 5]})
# basic plotly plot
data = [go.Bar(x=df['column_a'], y=df['column_b'])]
# instruction to view it inside jupyter
py.iplot(data, filename='jupyter-basic_bar')
# (alternatively) instruction to view it in web app of plotly
# py.plot(data)

In [8]:
# -- TEST 4 :
# verify you can use plotly and visualize plots in web browser locally

import plotly.io as pio            # to define input-output of plots
pio.renderers.default = "browser"  # to render the plot locally in your default web browser

# basic plotly plot
plot_data = go.Figure(go.Bar(x=df['column_a'], y=df['column_b']))
# instruction to view it in specified render (in this case browser)
plot_data.show()